Ce notebook présente le suivi et l’évaluation des modèles de classification appliqués à la prédiction du défaut de paiement.
Chaque modèle : Logistic Regression, Decision Tree et Random Forest est entraîné sur le même jeu de données et ses performances sont enregistrées avec MLflow.

L’objectif de ce notebook est de :

1) Démontrer la création et le suivi des experiments et runs MLflow pour chaque modèle
2) Sauvegarder et visualiser les métriques de performance (accuracy, F1-score, précision) et les artefacts (matrices de confusion, importances des features).
3) Identifier et sauvegarder le meilleur modèle selon le F1-score pour un usage ultérieur.

Avant de lancer ce notebook : assurez-vous que le serveur MLflow est démarré dans le terminal :

In [10]:
!mlflow server --host 127.0.0.1 --port 8080

/Users/bintoudiop/Documents/Dossier_École/Formation data analytics/MLOPS/Projet_MLOPS/venv/lib/python3.10/site-packages/mlflow/gateway/config.py:454: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class Route(ConfigModel):
ERROR:    [Errno 48] Address already in use
Running the mlflow server failed. Please see the logs above for details.


## 1. Import des librairies

In [41]:
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    confusion_matrix, ConfusionMatrixDisplay, roc_auc_score
)
from sklearn.pipeline import Pipeline
from mlflow.models.signature import infer_signature
import mlflow
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline as ImbPipeline
from mlflow.tracking import MlflowClient
from sklearn.preprocessing import StandardScaler

In [42]:
# Connexion au serveur MLflow local
mlflow.set_tracking_uri("http://127.0.0.1:8080")
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")
print("Connecté au serveur MLflow local : http://127.0.0.1:8080")

Connecté au serveur MLflow local : http://127.0.0.1:8080


## 2. Configuration des chemins

In [43]:
BASE_DIR = os.getcwd()
DATA_PATH = os.path.join(BASE_DIR, "data", "loan_data_preprocessed.csv")
MODELS_DIR = os.path.join(BASE_DIR, "Models")
MLARTIFACTS_DIR = os.path.join(BASE_DIR, "mlartifacts")

os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(MLARTIFACTS_DIR, exist_ok=True)

print(f"Répertoires prêts : {BASE_DIR}, {MODELS_DIR}, {MLARTIFACTS_DIR}")

Répertoires prêts : /Users/bintoudiop/Documents/Dossier_École/Formation data analytics/MLOPS/Projet_MLOPS, /Users/bintoudiop/Documents/Dossier_École/Formation data analytics/MLOPS/Projet_MLOPS/Models, /Users/bintoudiop/Documents/Dossier_École/Formation data analytics/MLOPS/Projet_MLOPS/mlartifacts


## 3. Chargement et préparation des données

In [44]:
df = pd.read_csv(DATA_PATH)
df.head()

# Séparation features / target
X = df[['credit_lines_outstanding', 'loan_amt_outstanding',
          'total_debt_outstanding', 'income', 'years_employed', 'fico_score']]
y = df["default"]
X = X.apply(pd.to_numeric, errors='coerce').astype("float64")

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)
print(f"Train: {X_train.shape}, Test: {X_test.shape}")

Train: (8000, 6), Test: (2000, 6)


In [45]:
# Les données sont désiquilibrées, on équilibre les classes
print("\n Vérification du déséquilibre ")
counter = Counter(y_train)
print("Avant rééchantillonnage :", counter)

ros = RandomOverSampler(random_state=42)
X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

counter_res = Counter(y_train_res)
print("Après rééchantillonnage :", counter_res)



 Vérification du déséquilibre 
Avant rééchantillonnage : Counter({0: 6519, 1: 1481})
Après rééchantillonnage : Counter({1: 6519, 0: 6519})


## 4. Définition des modèles

In [46]:
models = {
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "RandomForest": RandomForestClassifier(random_state=42)
}

best_f1 = 0
best_model = None
best_name = ""

## 5. Entraînement et tracking MLflow

In [47]:
for name, model in models.items():

    artifact_path = os.path.join(MLARTIFACTS_DIR, name)
    os.makedirs(artifact_path, exist_ok=True)


    try:
        experiment_id = client.create_experiment(
            name=name,
            artifact_location=artifact_path
        )
    except mlflow.exceptions.MlflowException:
        experiment = client.get_experiment_by_name(name)
        experiment_id = experiment.experiment_id

    mlflow.set_experiment(name)


    with mlflow.start_run(run_name=f"{name}_run"):

        # --- Pipeline complet ---
        pipeline = ImbPipeline([
            ('scaler', StandardScaler()),                 # Scale les données
            ('oversample', RandomOverSampler(random_state=42)),  # Équilibre les classes
            ('classifier', model)
        ])
        pipeline.fit(X_train, y_train)

        #  Prédictions
        y_train_pred = pipeline.predict(X_train)
        y_test_pred = pipeline.predict(X_test)

        #  Métriques train (console uniquement)
        acc_train = accuracy_score(y_train, y_train_pred)
        f1_train = f1_score(y_train, y_train_pred)
        prec_train = precision_score(y_train, y_train_pred)
        recall_train = recall_score(y_train, y_train_pred)
        tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, y_train_pred).ravel()
        print(f"\n {name} : Métriques train ")
        print(f"Accuracy: {acc_train:.3f}, F1: {f1_train:.3f}, Precision: {prec_train:.3f}, Recall: {recall_train:.3f}")
        print(f"Vrais positifs: {tp_train}, Vrais négatifs: {tn_train}, Faux positifs: {fp_train}, Faux négatifs: {fn_train}")
        if abs(f1_train - f1_score(y_test, y_test_pred)) > 0.05:
            print(" Signe possible d'overfitting")
        else:
            print(" Pas de signe d'overfitting évident")

        #  Métriques test (logging MLflow)
        acc_test = accuracy_score(y_test, y_test_pred)
        f1_test = f1_score(y_test, y_test_pred)
        prec_test = precision_score(y_test, y_test_pred)
        recall_test = recall_score(y_test, y_test_pred)
        tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_test_pred).ravel()
        fpr_test = fp_test / (fp_test + tn_test)
        fnr_test = fn_test / (fn_test + tp_test)
        roc_auc_test = roc_auc_score(y_test, pipeline.predict_proba(X_test)[:,1]) if hasattr(model, "predict_proba") else None

        mlflow.log_param("model_name", name)
        mlflow.log_metric("accuracy", acc_test)
        mlflow.log_metric("f1_score", f1_test)
        mlflow.log_metric("precision", prec_test)
        mlflow.log_metric("recall_test", recall_test)
        mlflow.log_metric("vrais_positifs", tp_test)
        mlflow.log_metric("vrais_negatifs", tn_test)
        mlflow.log_metric("faux_positifs", fp_test)
        mlflow.log_metric("faux_negatifs", fn_test)
        mlflow.log_metric("taux_faux_positifs", fpr_test)
        mlflow.log_metric("taux_faux_negatifs", fnr_test)
        if roc_auc_test is not None:
            mlflow.log_metric("roc_auc", roc_auc_test)

        # Matrice de confusion
        cm = confusion_matrix(y_test, y_test_pred)
        fig, ax = plt.subplots(figsize=(6,5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
        plt.xlabel("Prédit")
        plt.ylabel("Réel")
        plt.title(f"Matrice de confusion - {name}")
        plt.tight_layout()
        cm_file = os.path.join(artifact_path, f"confusion_matrix_{name}.png")
        plt.savefig(cm_file)
        plt.close()
        mlflow.log_artifact(cm_file)


        if hasattr(model, "feature_importances_"):
            importances = model.feature_importances_
            fig, ax = plt.subplots(figsize=(7,5))
            sns.barplot(x=X.columns, y=importances, ax=ax)
            plt.xticks(rotation=45)
            plt.title(f"Feature Importance - {name}")
            plt.tight_layout()
            feat_file = os.path.join(artifact_path, f"feature_importance_{name}.png")
            plt.savefig(feat_file)
            plt.close()
            mlflow.log_artifact(feat_file)

        #  Sauvegarde meilleur modèle
        if f1_test > best_f1:
            best_f1 = f1_test
            best_model = pipeline
            best_name = name

            model_file = os.path.join(MODELS_DIR, f"BestModel_{name}_{best_f1:.4f}.pkl")
            with open(model_file, "wb") as f:
                pickle.dump(best_model, f)
            print(f"-> Nouveau meilleur modèle sauvegardé : {model_file}")

        #  Logging modèle complet
        input_example = X_test.iloc[:1]
        signature = infer_signature(X_test, pipeline.predict(X_test))
        mlflow.sklearn.log_model(
            sk_model=pipeline,
            name="model",
            signature=signature,
            input_example=input_example
        )



 LogisticRegression : Métriques train 
Accuracy: 0.995, F1: 0.987, Precision: 0.974, Recall: 1.000
Vrais positifs: 1481, Vrais négatifs: 6479, Faux positifs: 40, Faux négatifs: 0
 Pas de signe d'overfitting évident
-> Nouveau meilleur modèle sauvegardé : /Users/bintoudiop/Documents/Dossier_École/Formation data analytics/MLOPS/Projet_MLOPS/Models/BestModel_LogisticRegression_0.9920.pkl
🏃 View run LogisticRegression_run at: http://127.0.0.1:8080/#/experiments/779486009998673081/runs/6e7099ab338c4ae49b7bc25ffc23a56a
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/779486009998673081

 DecisionTree : Métriques train 
Accuracy: 1.000, F1: 1.000, Precision: 1.000, Recall: 1.000
Vrais positifs: 1481, Vrais négatifs: 6519, Faux positifs: 0, Faux négatifs: 0
 Pas de signe d'overfitting évident
🏃 View run DecisionTree_run at: http://127.0.0.1:8080/#/experiments/326855897572357880/runs/e5fe57974f004a0eb93a0a564b5a58ac
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/3268558975

## 6. Résultat final

In [48]:
print(f"Meilleur modèle : {best_name} avec F1-score = {best_f1:.4f}")

Meilleur modèle : LogisticRegression avec F1-score = 0.9920


Chaque modèle a été suivi et évalué avec MLflow avec métriques et artefacts centralisés.
Le meilleur modèle, sélectionné selon le F1-score est la Logistic Regression qui présente les performances les plus équilibrées sur le jeu de test.